In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import callbacks
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,Dropout
import matplotlib.pyplot as plt
from sklearn import metrics
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

import keras
from keras import *
from keras.layers import *
from pythainlp import word_tokenize
from pythainlp.word_vector import *

from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('Suicidal_K5_Train.csv').drop(columns='Unnamed: 0')

In [3]:
df

,DateTime,Uid,Tweet,Label (Specialist),HashTag
0,2021-05-18 7:24:39,a38755f80aee,ถ้าฉันตายไปจริงๆก็คงดีสินะ,Level 2,#โรคซึมเศร้า
1,2021-05-20 16:06:12,a38755f80aee,เหนื่อยเหนื่อยจนจะตายอยู่แล้วมันไมกันนะนั่งเล่...,Level 1,#โรคซึมเศร้า
2,2021-06-16 7:00:59,9b7d113b270a,พยายามที่จะมีชีวิตอยู่แต่ก็เหนื่อยกับโลกนี้เหล...,Level 1,#โรคซึมเศร้า
3,2021-05-28 18:43:41,0b981040c16d,ชีวิตก็มีอยู่แค่นี้อยากหายไปแล้วนะ,Level 1,#โรคซึมเศร้า
4,2021-07-13 18:06:11,937c5794646d,วันวุ่นๆของวัยรุ่นกินยาเกินขนาดกรีดแขนพร้อม,Level 5,#โรคซึมเศร้า
...,...,...,...,...,...
4616,2021-05-14 18:30:04,effe3c2573da,อยู่มาได้ขนาดนี้ถือว่าเก่งแล้ว,Other,#โรคซึมเศร้า
4617,2021-05-14 18:30:04,effe3c2573da,อยู่มาได้ขนาดนี้ถือว่าเก่งแล้ว,Other,#โรคซึมเศร้า
4618,2021-09-28 12:23:17,f9325bb44007,จะพยายามไม่ทำร้ายตัวเองในวันที่ดิ่ง,Other,#โรคซึมเศร้า
4619,2021-09-25 1:28:50,fb395b09645c,แล้วเจอกันนะ,Other,#ครอบครัวไม่ใช่เซฟโซน #เหนื่อยกับชีวิต #อยากตา...


In [4]:
wModel = get_model()
thai2dict = {}
for word in wModel.index_to_key:
    thai2dict[word] = wModel[word]
thai2vec = pd.DataFrame.from_dict(thai2dict,orient='index')
thVocab = thai2vec.index.to_list()

In [5]:
thai2vec.head(10)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
ที่,0.308956,-0.097699,0.116745,0.215612,0.015768,-0.064163,0.062168,0.039649,0.864940,0.846904,...,-0.142418,0.033241,0.171581,-0.624864,-0.009358,0.449131,0.120130,-0.122195,-0.450617,-0.071318
และ,0.010751,-0.618971,0.129665,0.035460,-0.007560,0.027607,0.397824,0.026543,0.254075,0.168328,...,-0.105786,0.180930,-0.101630,0.070885,-0.037263,0.183606,-0.049088,-0.672288,-1.293044,0.592576
เป็น,-0.015736,-0.258926,0.052953,0.153728,-0.005985,-0.021081,0.041088,0.057312,1.633230,0.442729,...,-0.009408,-0.252576,-0.305512,0.372542,0.049151,0.568470,0.266586,0.400800,-0.784650,0.197369
ของ,-0.189711,-0.174774,0.171124,-0.186771,0.054294,-0.114150,-1.109456,-0.094466,-0.447015,0.042377,...,-0.168676,-0.148738,0.680404,0.097702,0.020270,0.182967,-0.083949,0.006287,-0.707434,-0.070234
มี,-0.156962,-0.231863,0.080312,0.323157,0.215695,0.055145,0.420794,0.016842,0.256759,0.832864,...,-0.044267,-0.147186,-0.105424,0.907078,0.009299,0.550953,0.139337,0.031696,-0.670379,-0.008048
ได้,-0.428813,-0.031194,0.041922,-0.036608,-0.008106,0.076470,-0.782270,0.033361,0.606864,0.440520,...,0.024458,-0.025031,0.103389,-0.078255,0.034323,0.459774,-0.748643,0.337775,-0.487408,-0.511535
"""""""""",-0.287710,0.064193,0.205076,0.146356,-0.071343,-0.039451,-1.845461,0.163763,1.018096,0.272786,...,0.051024,-0.532856,-0.131856,-0.090323,-0.058895,0.151262,-0.420358,0.055971,-0.930814,0.163908
การ,0.239587,-0.303620,0.079953,-0.453045,-0.528826,-0.161692,0.235725,-0.099673,0.691668,0.536159,...,-0.110436,-0.297495,-0.217414,0.045158,0.066647,0.190095,-0.304333,-0.724927,-0.995488,-0.716609
(,-0.120522,-0.355783,0.168180,-0.377733,-0.158624,-0.047249,0.361140,0.161460,0.913314,0.345037,...,0.116285,-0.318218,-0.356664,0.519889,0.130475,0.125772,0.101328,-0.382658,-1.205359,0.340139
),-0.086848,-0.155231,0.133015,-0.039913,0.183761,0.115142,-1.940854,-0.066565,-2.399744,0.146722,...,0.019406,-0.181474,0.099863,0.516092,0.201697,0.249139,0.252957,1.138815,-0.018209,0.232265


In [6]:
ll = len(thai2vec)


In [7]:
for vidx in range(ll):
    if vidx % 100 == 0:
        print('\r' + str(vidx),end='')
    aa = thai2vec.iloc[[vidx]]
    ab = aa.values.tolist()
    if vidx == 0:
        vect = ab
    else:
        vect = np.vstack((vect,ab))

print("\n", vect.shape)

51300
 (51358, 300)


In [8]:
def convert2num(convData):
    for ci in range(len(convData)):
        if convData[ci] == "Level 1":
            convData[ci] = 0
        elif convData[ci] == "Level 2":
            convData[ci] = 1
        elif convData[ci] == "Level 3":
            convData[ci] = 2
        elif convData[ci] == "Level 4":
            convData[ci] = 3
        elif convData[ci] == "Level 5":
            convData[ci] = 4
        elif convData[ci] == "Other":
            convData[ci] = 5
    return convData

In [9]:
def tokenWord(wordTarget):
    wordToken = word_tokenize(wordTarget, engine='attacut')
    return wordToken

In [10]:
def convWord(cw):
    cWord = cw
    for ti in range(len(cWord)):
        if cWord[ti] == ' ':
            cWord[ti] = ''
        elif cWord[ti] not in thVocab:
            cWord[ti] = ''
    return cWord

In [11]:
def token2index(t2idx):
    w2index = []
    for wi in range(len(t2idx)):
        if t2idx[wi] in thVocab:
            w2index.append(thVocab.index(t2idx[wi]))
    return np.array(w2index)

In [12]:
def findMaxArray(fma):
    maxlen = 0
    for mi in range(len(fma)):
        nA = len(fma[mi])
        if nA > maxlen:
            maxlen = nA
    return maxlen

In [13]:
def fill0in(f0i):
    fMax = findMaxArray(f0i)
    for fi, ax in enumerate(f0i):
        if len(ax) < fMax:
            f0i[fi] = np.hstack((ax , np.zeros(fMax-len(ax))))
        f0i1 = np.array(f0i)
    return f0i1

In [14]:
def prepare2train(ipt):
    pre2t = []
    for pidx in range(len(ipt)):
        wp1 = ipt[pidx]
        pre2t.append(token2index(convWord(tokenWord(wp1))))
    return pre2t

In [15]:
tweet = df['Tweet']
tweetArray = tweet.to_list()
label = df['Label (Specialist)']

In [16]:
Xtrain = prepare2train(tweetArray)
Ytrain = convert2num(label.to_list())

In [17]:
YTrainIdx = np.array(Ytrain)
XTrainIdx = fill0in(Xtrain)

/var/folders/y8/1g3ffxtj5bl6btf8w5y73r_00000gn/T/ipykernel_74864/4258061065.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  f0i1 = np.array(f0i)


In [18]:
XTrainIdx.shape

(4621, 79)

In [33]:
model = keras.Sequential()
model.add(keras.layers.Embedding(vect.shape[0],vect.shape[1],name='embed'))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64)))
model.add(Dropout(0.1))
model.add(Dense(32, activation=('relu')))
model.add(Dense(6, activation=('softmax')))

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed (Embedding)           (None, None, 300)         15407400  
                                                                 
 bidirectional_2 (Bidirectio  (None, None, 256)        439296    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 32)                4128      
                                                                 
 dense_3 (Dense)             (None, 6)                

In [35]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [36]:
model.get_layer('embed').set_weights([vect])
model.get_layer('embed').trainable = True

In [ ]:
class myCallback(callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95):
      self.model.stop_training = True
      print("\nCallback Accuracy มากกว่า 95%!")
callbacks = myCallback()

In [38]:
model.fit(XTrainIdx,YTrainIdx,epochs=20,batch_size=128,callbacks=[callbacks])

Epoch 1/20


2022-08-10 02:04:20.385928: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-10 02:04:21.038901: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-10 02:04:21.054741: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-10 02:04:21.284586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-10 02:04:21.304181: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-10 02:04:21.567930: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-10 02:04:21.591916: I tensorflow/core/grappler/optimizers/cust

37/37 [==============================] - 13s 223ms/step - loss: 1.3639 - accuracy: 0.5337
Epoch 2/20
37/37 [==============================] - 7s 202ms/step - loss: 1.1480 - accuracy: 0.5981
Epoch 3/20
37/37 [==============================] - 7s 196ms/step - loss: 1.0037 - accuracy: 0.6581
Epoch 4/20
37/37 [==============================] - 7s 193ms/step - loss: 0.9090 - accuracy: 0.6923
Epoch 5/20
37/37 [==============================] - 7s 195ms/step - loss: 0.8343 - accuracy: 0.7224
Epoch 6/20
37/37 [==============================] - 7s 195ms/step - loss: 0.7581 - accuracy: 0.7433
Epoch 7/20
37/37 [==============================] - 7s 195ms/step - loss: 0.6926 - accuracy: 0.7691
Epoch 8/20
37/37 [==============================] - 7s 196ms/step - loss: 0.6390 - accuracy: 0.7816
Epoch 9/20
37/37 [==============================] - 7s 194ms/step - loss: 0.5897 - accuracy: 0.8029
Epoch 10/20
37/37 [==============================] - 7s 194ms/step - loss: 0.5295 - accuracy: 0.8243
Epoch 11/

In [25]:
test_df = pd.read_csv('Suicidal_K5_Test.csv').drop(columns='Unnamed: 0')
test_label = test_df['Label (Specialist)']
X_test = test_df['Tweet']

test_array = X_test.to_list()

X_test = prepare2train(test_array)
y_test = convert2num(test_label)

In [26]:
YTestIdx = np.array(y_test)
XTestIdx = fill0in(X_test)

/var/folders/y8/1g3ffxtj5bl6btf8w5y73r_00000gn/T/ipykernel_74864/4258061065.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  f0i1 = np.array(f0i)


In [27]:
y_pred = np.argmax(model.predict(XTestIdx),axis=1)

2022-08-10 01:57:33.700191: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-10 01:57:33.901174: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-10 01:57:33.914345: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-10 01:57:34.041863: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-10 01:57:34.052588: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [28]:
y_true = test_df['Label (Specialist)']
y_true

0      4
1      0
2      2
3      4
4      0
      ..
509    5
510    5
511    5
512    5
513    5
Name: Label (Specialist), Length: 514, dtype: object

In [29]:
XTestIdx = np.asarray(XTestIdx).astype(np.float32)
YTestIdx = np.asarray(YTestIdx).astype(np.float32)

In [30]:
accr = model.evaluate(XTestIdx,YTestIdx)

2022-08-10 01:57:35.459894: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-10 01:57:35.680203: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-10 01:57:35.690657: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-10 01:57:35.816412: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-10 01:57:35.827000: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


17/17 [==============================] - 2s 49ms/step - loss: 1.2666 - accuracy: 0.6479


In [31]:
y_true = y_true.tolist()
y_pred = y_pred.tolist()

In [32]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.85      0.82       277
           1       0.33      0.32      0.32        50
           2       0.41      0.29      0.34        24
           3       0.31      0.22      0.26        36
           4       0.51      0.56      0.54        66
           5       0.57      0.49      0.53        61

    accuracy                           0.65       514
   macro avg       0.49      0.46      0.47       514
weighted avg       0.63      0.65      0.64       514

